# Best place for fast food venue in Moscow.

## Introduction.
In this research we will try to find an optimal lication for a fast food venue in Moscow, Russia.
We will try to detect locations that are less crowded with restaurants and potentially have the largest amount of potential clients.
We target stakeholders who are interested in starting fastfood buisness in Moscow. For those we will show few most perspective neibourhoods and express the advantages of each of them, so that best possible final location can be chosen by stakeholders.

## Data.
It is well known that neibourhoods near Moscow Subway (Metro) stations have the biggest traffic of potential clients of fastfood restraurants in the city. That's wy we will prepare dataset of all  Metro stations (more than 300 stations) and analise neibourhoods around them. Average distance between two stations is about 2 kilometers, so we will explore areas within 1 kilometer from stations. To prepare this data we will:
1. scrap wikipedia page to get the list of stations;
2. use geopy library to get their latitude and longitude.

Next we will obtain dataset of all venues of catigories we are interested in and search for locations less crowded with fastfood restaurants. To prepare this data we will use Forsquare API.

Let's import libraryes we will need.

In [2]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import json
!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim
import requests
from bs4 import BeautifulSoup
from pandas.io.json import json_normalize
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
!conda install -c conda-forge folium=0.5.0 --yes
import folium

Solving environment: ...working... 
  - anaconda/win-64::ca-certificates-2019.8.28-0, anaconda/win-64::openssl-1.1.1d-he774522_2
  - anaconda/win-64::ca-certificates-2019.8.28-0, defaults/win-64::openssl-1.1.1d-he774522_2
  - anaconda/win-64::openssl-1.1.1d-he774522_2, defaults/win-64::ca-certificates-2019.8.28-0
  - defaults/win-64::ca-certificates-2019.8.28-0, defaults/win-64::openssl-1.1.1d-he774522_2done

# All requested packages already installed.

Solving environment: ...working... 
  - anaconda/win-64::ca-certificates-2019.8.28-0, anaconda/win-64::openssl-1.1.1d-he774522_2
  - anaconda/win-64::openssl-1.1.1d-he774522_2, defaults/win-64::ca-certificates-2019.8.28-0
  - anaconda/win-64::ca-certificates-2019.8.28-0, defaults/win-64::openssl-1.1.1d-he774522_2
  - defaults/win-64::ca-certificates-2019.8.28-0, defaults/win-64::openssl-1.1.1d-he774522_2done

# All requested packages already installed.

Libraries imported.


Now we need to prepare dataset of Moscow Metro stations containing name of the station it's latitude and longitude.
Luckely there is a page on Wikipedia whith a list of all Metro stations. All we need is to scrap the page and wrangle data, so it can be used further in the resirch.

In [3]:
req = requests.get("https://en.wikipedia.org/wiki/List_of_Moscow_Metro_stations")
soup = BeautifulSoup(req.content,'lxml')
table = soup.find_all('table')[1]
df = pd.read_html(str(table))
nb=pd.DataFrame(df[0])

In [4]:
nb.drop(columns=['L','Russian Cyrillic','Transfer','Opened','Elev.','Type','Pic.'], axis=1, inplace=True)

lat=[]
lon=[]
for i in range(261):
    lat.append(nb.loc[i][1][26:33])
    lon.append(nb.loc[i][1][36:43])
    if lon[i][0] == 'N':
        lon[i] = nb.loc[i][1][38:45]
nb['Lat'] = lat
nb['Lon'] = lon

nb.drop(columns='Coordinates',axis=1, inplace=True)

nb.columns=['Name','Latitude','Longitude']

nb['Latitude']=nb['Latitude'].astype(float)

nb['Longitude']=nb['Longitude'].astype(float)

nb.head()

,Name,Latitude,Longitude
0,Bulvar Rokossovskogo,55.8148,37.7342
1,Cherkizovskaya,55.8038,37.7448
2,Preobrazhenskaya Ploshchad,55.7963,37.7151
3,Sokolniki,55.7888,37.6802
4,Krasnoselskaya,55.7801,37.6673


Now let's visualize our dataset using folium.
First we need to get coordinates of Moscow to center the map. Let's take famouus Red Square as a center of Moscow and get it's coordinates using geopy library.

In [6]:
address = 'Red Square, Moscow'

geolocator = Nominatim(user_agent="ny_explorer")
RS_location = geolocator.geocode(address)
RS_latitude = RS_location.latitude
RS_longitude = RS_location.longitude
print('The geograpical coordinate of Red Square, Moscow are {}, {}.'.format(RS_latitude, RS_longitude))

The geograpical coordinate of Red Square, Moscow are 55.7536283, 37.62137960067377.


Now let's create a map of Moscow with Metro stations located on it as blue circles. Red square will be marked as the red circle in the center of the map.

In [7]:
metro_map = folium.Map(location=[RS_latitude, RS_longitude], zoom_start=10)

# add a red circle marker to represent the Red Square
folium.features.CircleMarker(
    [RS_latitude, RS_longitude],
    radius=10,
    color='red',
    popup='Red Square',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(metro_map)

# add the Metro stations as blue circle markers
for lat, lng, label in zip(nb.Latitude, nb.Longitude, nb.Name):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(metro_map)

# display map
metro_map

Size of Moscow Metro is quite impressive. Now we need to prepare a dataset of venues ner each station. For this we will use API of a popular database "Forsquare".

API credentials. Since we use a sandbox account I leave credentials open, so anyone who is interested can use them. Limit of venues by request is set to 100, and category we are looking for is the root categori ID for food venues provided by Foursquare.

In [240]:
CLIENT_ID = '2J2XW3UMQSA4HQVAEHSPF4S3U0KJCIZ4PSGEGKBBDARFRABX'
CLIENT_SECRET = 'OZIL1STXTN4H3GUDXKCAXRK2NDKSP1DJANQPIIS0PPNTNKWN'
VERSION = '20200223'

LIMIT=100
category = '4d4b7105d754a06374d81259'

Here is the function that takes name of station and it's coordinates and returns a dataset of nearby venues.

In [196]:
def getNearbyVenues(names, latitudes, longitudes, radius=1000):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&categoryId={}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng,
            category,
            radius, 
            LIMIT)
            
        try:
            results = requests.get(url).json()["response"]['groups'][0]['items']
        except:
            try:
                results = requests.get(url).json()["response"]['groups'][0]['items']
            except:
                venues_list.append([(name,lat,lng,np.nan,np.nan,np.nan,np.nan)])
        
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

Now we use the funtction with our Metro Stations dataset.

In [197]:
metro_venues = getNearbyVenues(names=nb['Name'],
                                   latitudes=nb['Latitude'],
                                   longitudes=nb['Longitude']
                                  )

Bulvar Rokossovskogo
Cherkizovskaya
Preobrazhenskaya Ploshchad
Sokolniki
Krasnoselskaya
Komsomolskaya
Krasnye Vorota
Chistyye Prudy
Lubyanka
Okhotny Ryad
Biblioteka Imeni Lenina
Kropotkinskaya
Park Kultury
Frunzenskaya
Sportivnaya
Vorobyovy Gory
Universitet
Prospekt Vernadskogo
Yugo-Zapadnaya
Troparyovo
Rumyantsevo
Salaryevo
Khovrino
Belomorskaya
Rechnoy Vokzal
Vodny Stadion
Voykovskaya
Sokol
Aeroport
Dinamo
Belorusskaya
Mayakovskaya
Tverskaya
Teatralnaya
Novokuznetskaya
Paveletskaya
Avtozavodskaya
Tekhnopark
Kolomenskaya
Kashirskaya
Kantemirovskaya
Tsaritsyno
Orekhovo
Domodedovskaya
Krasnogvardeyskaya
Alma-Atinskaya
Pyatnitskoye Shosse
Mitino
Volokolamskaya
Myakinino
Strogino
Krylatskoye
Molodyozhnaya
Kuntsevskaya
Slavyansky Bulvar
Park Pobedy
Kiyevskaya
Smolenskaya
Arbatskaya
Ploshchad Revolyutsii
Kurskaya
Baumanskaya
Elektrozavodskaya
Semyonovskaya
Partizanskaya
Izmaylovskaya
Pervomayskaya
Shchyolkovskaya
Kuntsevskaya
Pionerskaya
Filyovsky Park
Bagrationovskaya
Fili
Kutuzovskaya
Stu

As a result we got a dataset of more than 12000 venues. Let's have a closer look on it.

In [199]:
metro_venues.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12378 entries, 0 to 12377
Data columns (total 7 columns):
Neighborhood              12378 non-null object
Neighborhood Latitude     12378 non-null float64
Neighborhood Longitude    12378 non-null float64
Venue                     12378 non-null object
Venue Latitude            12378 non-null float64
Venue Longitude           12378 non-null float64
Venue Category            12378 non-null object
dtypes: float64(4), object(3)
memory usage: 677.0+ KB


In [200]:
metro_venues['Neighborhood'].value_counts()

Kiyevskaya                     271
Paveletskaya                   200
Kitay-gorod                    200
Kurskaya                       200
Belorusskaya                   200
Tretyakovskaya                 200
Taganskaya                     200
Arbatskaya                     200
Smolenskaya                    200
Oktyabrskaya                   188
Komsomolskaya                  177
Park Kultury                   175
Prospekt Mira                  167
Savyolovskaya                  158
Delovoy Tsentr                 144
Kutuzovskaya                   144
Avtozavodskaya                 132
Kuznetsky Most                 100
Novoslobodskaya                100
Chkalovskaya                   100
Serpukhovskaya                 100
Sretensky Bulvar               100
Turgenevskaya                  100
Dostoyevskaya                  100
Okhotny Ryad                   100
Trubnaya                       100
Marksistskaya                  100
Pushkinskaya                   100
Mendeleyevskaya     

It is interesting to look at all categories of venues and check the most popular for each station.

In [201]:
metro_onehot = pd.get_dummies(metro_venues[['Venue Category']], prefix="", prefix_sep="")
metro_onehot.head()
metro_onehot['Name'] = metro_venues['Neighborhood'] 
fixed_columns = [metro_onehot.columns[-1]] + list(metro_onehot.columns[:-1])
metro_onehot = metro_onehot[fixed_columns]

print(metro_onehot.shape)
metro_onehot.head()

(12378, 109)


,Name,American Restaurant,Arepa Restaurant,Argentinian Restaurant,Asian Restaurant,BBQ Joint,Bagel Shop,Bakery,Belarusian Restaurant,Belgian Restaurant,Bistro,Blini House,Brasserie,Brazilian Restaurant,Breakfast Spot,Buffet,Bulgarian Restaurant,Burger Joint,Burrito Place,Cafeteria,Café,Cantonese Restaurant,Caucasian Restaurant,Chinese Restaurant,Comfort Food Restaurant,Creperie,Cuban Restaurant,Czech Restaurant,Deli / Bodega,Dim Sum Restaurant,Diner,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,English Restaurant,Ethiopian Restaurant,Falafel Restaurant,Fast Food Restaurant,Fish & Chips Shop,Food,Food Court,Food Stand,Food Truck,French Restaurant,Fried Chicken Joint,Gastropub,German Restaurant,Greek Restaurant,Halal Restaurant,Hot Dog Joint,Indian Chinese Restaurant,Indian Restaurant,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Restaurant,Jewish Restaurant,Kebab Restaurant,Korean Restaurant,Kosher Restaurant,Latin American Restaurant,Lebanese Restaurant,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Modern European Restaurant,Molecular Gastronomy Restaurant,Moroccan Restaurant,New American Restaurant,Noodle House,Paella Restaurant,Pakistani Restaurant,Pelmeni House,Persian Restaurant,Peruvian Restaurant,Pet Café,Pizza Place,Poke Place,Ramen Restaurant,Restaurant,Russian Restaurant,Salad Place,Sandwich Place,Scandinavian Restaurant,Seafood Restaurant,Shawarma Place,Snack Place,Soup Place,South American Restaurant,Souvlaki Shop,Spanish Restaurant,Steakhouse,Sushi Restaurant,Swiss Restaurant,Taco Place,Tapas Restaurant,Tatar Restaurant,Tex-Mex Restaurant,Thai Restaurant,Theme Restaurant,Trattoria/Osteria,Turkish Restaurant,Udon Restaurant,Ukrainian Restaurant,Varenyky restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wings Joint
0,Bulvar Rokossovskogo,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Bulvar Rokossovskogo,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
2,Bulvar Rokossovskogo,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Bulvar Rokossovskogo,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
4,Bulvar Rokossovskogo,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [202]:
metro_grouped = metro_onehot.groupby('Name').mean().reset_index()
print(metro_grouped.shape)
metro_grouped.head()

(232, 109)


,Name,American Restaurant,Arepa Restaurant,Argentinian Restaurant,Asian Restaurant,BBQ Joint,Bagel Shop,Bakery,Belarusian Restaurant,Belgian Restaurant,Bistro,Blini House,Brasserie,Brazilian Restaurant,Breakfast Spot,Buffet,Bulgarian Restaurant,Burger Joint,Burrito Place,Cafeteria,Café,Cantonese Restaurant,Caucasian Restaurant,Chinese Restaurant,Comfort Food Restaurant,Creperie,Cuban Restaurant,Czech Restaurant,Deli / Bodega,Dim Sum Restaurant,Diner,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,English Restaurant,Ethiopian Restaurant,Falafel Restaurant,Fast Food Restaurant,Fish & Chips Shop,Food,Food Court,Food Stand,Food Truck,French Restaurant,Fried Chicken Joint,Gastropub,German Restaurant,Greek Restaurant,Halal Restaurant,Hot Dog Joint,Indian Chinese Restaurant,Indian Restaurant,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Restaurant,Jewish Restaurant,Kebab Restaurant,Korean Restaurant,Kosher Restaurant,Latin American Restaurant,Lebanese Restaurant,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Modern European Restaurant,Molecular Gastronomy Restaurant,Moroccan Restaurant,New American Restaurant,Noodle House,Paella Restaurant,Pakistani Restaurant,Pelmeni House,Persian Restaurant,Peruvian Restaurant,Pet Café,Pizza Place,Poke Place,Ramen Restaurant,Restaurant,Russian Restaurant,Salad Place,Sandwich Place,Scandinavian Restaurant,Seafood Restaurant,Shawarma Place,Snack Place,Soup Place,South American Restaurant,Souvlaki Shop,Spanish Restaurant,Steakhouse,Sushi Restaurant,Swiss Restaurant,Taco Place,Tapas Restaurant,Tatar Restaurant,Tex-Mex Restaurant,Thai Restaurant,Theme Restaurant,Trattoria/Osteria,Turkish Restaurant,Udon Restaurant,Ukrainian Restaurant,Varenyky restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wings Joint
0,Aeroport,0.00,0.0,0.0,0.021739,0.000000,0.0,0.108696,0.0,0.0,0.021739,0.021739,0.00,0.0,0.000000,0.00,0.00,0.021739,0.0,0.021739,0.217391,0.0,0.086957,0.0,0.000000,0.000000,0.0,0.0,0.043478,0.0,0.0,0.0,0.00,0.021739,0.000000,0.0,0.0,0.0,0.021739,0.0,0.0,0.000000,0.0,0.000000,0.0,0.021739,0.00,0.0,0.0,0.0,0.0,0.0,0.000000,0.021739,0.0,0.086957,0.043478,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.021739,0.0,0.0,0.043478,0.0,0.0,0.065217,0.00,0.0,0.021739,0.0,0.000000,0.021739,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.021739,0.021739,0.00,0.0,0.00,0.0
1,Akademicheskaya,0.00,0.0,0.0,0.000000,0.021277,0.0,0.212766,0.0,0.0,0.000000,0.000000,0.00,0.0,0.021277,0.00,0.00,0.000000,0.0,0.085106,0.170213,0.0,0.042553,0.0,0.021277,0.021277,0.0,0.0,0.000000,0.0,0.0,0.0,0.00,0.021277,0.042553,0.0,0.0,0.0,0.021277,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.00,0.0,0.0,0.0,0.0,0.0,0.021277,0.000000,0.0,0.042553,0.000000,0.0,0.0,0.021277,0.0,0.0,0.0,0.0,0.0,0.000000,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.021277,0.0,0.0,0.042553,0.00,0.0,0.021277,0.0,0.000000,0.021277,0.000000,0.0,0.0,0.0,0.0,0.000000,0.106383,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.000000,0.021277,0.00,0.0,0.00,0.0
2,Aleksandrovsky Sad,0.01,0.0,0.0,0.020000,0.020000,0.0,0.110000,0.0,0.0,0.010000,0.010000,0.01,0.0,0.010000,0.03,0.01,0.030000,0.0,0.000000,0.120000,0.0,0.060000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.02,0.010000,0.020000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.03,0.0,0.0,0.0,0.0,0.0,0.000000,0.030000,0.0,0.110000,0.020000,0.0,0.0,0.010000,0.0,0.0,0.0,0.0,0.0,0.000000,0.01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.030000,0.0,0.0,0.090000,0.03,0.0,0.030000,0.0,0.030000,0.010000,0.000000,0.0,0.0,0.0,0.0,0.020000,0.010000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.01,0.000000,0.010000,0.01,0.0,0.01,0.0
3,Alekseyevskaya,0.00,0.0,0.0,0.000000,0.000000,0.0,0.069767,0.0,0.0,0.000000,0.000000,0.00,0.0,0.023256,0.00,0.00,0.000000,0.0,0.162791,0.162791,0.0,0.023256,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.00,0.000000,0.093023,0.0,0.0,0.0,0.0

Let's look at top 5 venues for each location.

In [203]:
num_top_venues = 5

for hood in metro_grouped['Name']:
    print("----"+hood+"----")
    temp = metro_grouped[metro_grouped['Name'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Aeroport----
                  venue  freq
0                  Café  0.22
1                Bakery  0.11
2  Caucasian Restaurant  0.09
3    Italian Restaurant  0.09
4            Restaurant  0.07


----Akademicheskaya----
                         venue  freq
0                       Bakery  0.21
1                         Café  0.17
2             Sushi Restaurant  0.11
3                    Cafeteria  0.09
4  Eastern European Restaurant  0.04


----Aleksandrovsky Sad----
                  venue  freq
0                  Café  0.12
1    Italian Restaurant  0.11
2                Bakery  0.11
3            Restaurant  0.09
4  Caucasian Restaurant  0.06


----Alekseyevskaya----
                         venue  freq
0                         Café  0.16
1                    Cafeteria  0.16
2                  Pizza Place  0.09
3  Eastern European Restaurant  0.09
4                       Bakery  0.07


----Alma-Atinskaya----
                  venue  freq
0           Pizza Place  0.22
1  Fast Food R

                  venue  freq
0                Bakery  0.14
1                  Café  0.09
2  Caucasian Restaurant  0.09
3            Restaurant  0.05
4          Burger Joint  0.04


----Domodedovskaya----
                  venue  freq
0  Fast Food Restaurant  0.13
1                  Café  0.11
2            Restaurant  0.09
3      Sushi Restaurant  0.09
4           Blini House  0.06


----Dostoyevskaya----
                venue  freq
0                Café  0.11
1  Italian Restaurant  0.08
2              Bakery  0.08
3          Restaurant  0.08
4           Gastropub  0.06


----Dubrovka----
                       venue  freq
0                       Café  0.17
1                  Cafeteria  0.11
2       Fast Food Restaurant  0.07
3  Middle Eastern Restaurant  0.05
4      Vietnamese Restaurant  0.05


----Elektrozavodskaya----
                  venue  freq
0                  Café  0.20
1             Cafeteria  0.12
2            Restaurant  0.10
3  Fast Food Restaurant  0.10
4      Sushi Res

                 venue  freq
0            Cafeteria  0.29
1  Japanese Restaurant  0.14
2               Bistro  0.14
3          Snack Place  0.14
4                 Café  0.14


----Lokomotiv----
                  venue  freq
0  Fast Food Restaurant  0.15
1           Pizza Place  0.15
2                  Café  0.15
3             Cafeteria  0.15
4   Dumpling Restaurant  0.08


----Lomonosovsky Prospekt----
                venue  freq
0         Pizza Place  0.15
1  Chinese Restaurant  0.15
2                Café  0.15
3    Asian Restaurant  0.10
4              Bakery  0.10


----Lubyanka----
                  venue  freq
0  Caucasian Restaurant  0.10
1                  Café  0.10
2    Italian Restaurant  0.09
3          Burger Joint  0.05
4             Irish Pub  0.05


----Lukhmanovskaya----
                  venue  freq
0   Japanese Restaurant   0.2
1             BBQ Joint   0.2
2           Pizza Place   0.2
3  Caucasian Restaurant   0.2
4  Fast Food Restaurant   0.2


----Luzhniki----
   

                  venue  freq
0                  Café  0.18
1    Italian Restaurant  0.12
2  Fast Food Restaurant  0.12
3        Shawarma Place  0.12
4      Sushi Restaurant  0.06


----Petrovsky Park----
                         venue  freq
0                         Café  0.16
1                   Restaurant  0.11
2                    Cafeteria  0.11
3           Italian Restaurant  0.06
4  Eastern European Restaurant  0.06


----Pionerskaya----
                       venue  freq
0                Pizza Place  0.18
1  Middle Eastern Restaurant  0.18
2                 Restaurant  0.12
3                       Café  0.12
4       Fast Food Restaurant  0.12


----Planernaya----
                  venue  freq
0                  Café  0.24
1  Fast Food Restaurant  0.20
2            Restaurant  0.08
3            Food Court  0.08
4   Fried Chicken Joint  0.08


----Ploshchad Gagarina----
         venue  freq
0         Café  0.18
1       Bakery  0.10
2    Cafeteria  0.08
3   Restaurant  0.06
4  Piz

                         venue  freq
0                         Café  0.23
1           Italian Restaurant  0.06
2                  Pizza Place  0.06
3                Deli / Bodega  0.06
4  Eastern European Restaurant  0.06


----Sukharevskaya----
                       venue  freq
0         Italian Restaurant  0.08
1       Caucasian Restaurant  0.08
2                       Café  0.08
3                     Bakery  0.05
4  Middle Eastern Restaurant  0.05


----Sviblovo----
                  venue  freq
0            Restaurant  0.14
1           Pizza Place  0.11
2  Fast Food Restaurant  0.11
3    Belgian Restaurant  0.07
4      Sushi Restaurant  0.07


----Taganskaya----
                  venue  freq
0    Italian Restaurant  0.10
1            Restaurant  0.10
2                  Café  0.09
3                Bakery  0.08
4  Caucasian Restaurant  0.06


----Teatralnaya----
                venue  freq
0          Restaurant  0.09
1                Café  0.08
2  Italian Restaurant  0.06
3        B

                venue  freq
0          Restaurant  0.12
1                Café  0.12
2  Italian Restaurant  0.07
3          Steakhouse  0.04
4    Asian Restaurant  0.04


----Vystavochny Tsentr----
                       venue  freq
0                       Café  0.16
1                Pizza Place  0.10
2                  Cafeteria  0.06
3                 Restaurant  0.06
4  Middle Eastern Restaurant  0.06


----Yasenevo----
                  venue  freq
0  Fast Food Restaurant  0.24
1            Restaurant  0.10
2           Pizza Place  0.10
3             BBQ Joint  0.10
4                  Café  0.10


----Yugo-Zapadnaya----
                  venue  freq
0  Fast Food Restaurant  0.10
1             Cafeteria  0.08
2    Italian Restaurant  0.06
3           Pizza Place  0.06
4        Sandwich Place  0.06


----Yuzhnaya----
                         venue  freq
0                         Café  0.23
1         Fast Food Restaurant  0.15
2  Eastern European Restaurant  0.12
3                    B

Here is the function that returnes data sorted by most common categories of venues.

In [204]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Now we prepare a data set of top 10 categories for each station.

In [217]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

columns = ['Name']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

metro_venues_sorted = pd.DataFrame(columns=columns)
metro_venues_sorted['Name'] = metro_grouped['Name']

for ind in np.arange(metro_grouped.shape[0]):
    metro_venues_sorted.iloc[ind, 1:] = return_most_common_venues(metro_grouped.iloc[ind, :], num_top_venues)

metro_venues_sorted.head()

,Name,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Aeroport,Café,Bakery,Caucasian Restaurant,Italian Restaurant,Restaurant,Deli / Bodega,Japanese Restaurant,Pizza Place,Dumpling Restaurant,Sandwich Place
1,Akademicheskaya,Bakery,Café,Sushi Restaurant,Cafeteria,Eastern European Restaurant,Caucasian Restaurant,Italian Restaurant,Restaurant,Korean Restaurant,Fast Food Restaurant
2,Aleksandrovsky Sad,Café,Italian Restaurant,Bakery,Restaurant,Caucasian Restaurant,Pizza Place,Buffet,Irish Pub,Russian Restaurant,Sandwich Place
3,Alekseyevskaya,Cafeteria,Café,Pizza Place,Eastern European Restaurant,Bakery,Snack Place,Middle Eastern Restaurant,Italian Restaurant,Steakhouse,Fast Food Restaurant
4,Alma-Atinskaya,Pizza Place,Fast Food Restaurant,Italian Restaurant,Sushi Restaurant,Asian Restaurant,Food Truck,Café,Middle Eastern Restaurant,Restaurant,Bakery


As a result of data manipulation we have a dataset containing name of the Metro station, and 10 most common venues for each. This data we are going to analyse.

## Methodology

To find good locations for fast food venue we are going to divide the dataset by segments using popular machine learning сlustering algorithm k-means. Our goal is to bring some insights from results.

In [215]:
kclusters = 3

metro_grouped_clustering = metro_grouped.drop('Name', 1)

kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(metro_grouped_clustering)

kmeans.labels_[0:10] 

array([0, 0, 0, 2, 1, 1, 2, 1, 0, 2])

Let's merge initial dataset with cluster labels obtained by k-means.

In [218]:
metro_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

metro_merged = nb

metro_merged = metro_merged.join(metro_venues_sorted.set_index('Name'), on='Name')

metro_merged.head()

,Name,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Bulvar Rokossovskogo,55.8148,37.7342,1,Sushi Restaurant,Café,Vietnamese Restaurant,Asian Restaurant,Mexican Restaurant,Sandwich Place,Fried Chicken Joint,Fast Food Restaurant,Tex-Mex Restaurant,Food Court
1,Cherkizovskaya,55.8038,37.7448,2,Pizza Place,Fast Food Restaurant,Café,Cafeteria,Restaurant,Dumpling Restaurant,Bakery,Middle Eastern Restaurant,Burger Joint,Diner
2,Preobrazhenskaya Ploshchad,55.7963,37.7151,0,Sandwich Place,Bakery,Cafeteria,Caucasian Restaurant,Breakfast Spot,Sushi Restaurant,Italian Restaurant,Pizza Place,Eastern European Restaurant,Theme Restaurant
3,Sokolniki,55.7888,37.6802,2,Café,Restaurant,Fast Food Restaurant,Eastern European Restaurant,BBQ Joint,Pizza Place,Cafeteria,Noodle House,Indian Restaurant,Russian Restaurant
4,Krasnoselskaya,55.7801,37.6673,0,Cafeteria,Café,Fast Food Restaurant,Restaurant,Caucasian Restaurant,Snack Place,Fried Chicken Joint,Burger Joint,Vietnamese Restaurant,Bistro


In case of errors in сlustering process let's drop NA values.

In [219]:
metro_merged.dropna(axis=0,inplace=True)

## Results.

Let's visualize our results using folium.

In [220]:
map_clusters = folium.Map(location=[RS_latitude, RS_longitude], zoom_start=11)

x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

markers_colors = []
for lat, lon, poi, cluster in zip(metro_merged['Latitude'], metro_merged['Longitude'], metro_merged['Name'], metro_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster)-1],
        fill=True,
        fill_color=rainbow[int(cluster)-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

We got 3 klusters tending to form circles around the center of the city. Let's have a closer look at each of them and try to define it's meaning.

First cluster (red circles on the map above) mainly consists of restaurants of different cuisine. This kluster is the closest one to the city center.

In [221]:
print(metro_merged.loc[metro_merged['Cluster Labels'] == 0, metro_merged.columns[[0] + list(range(4, metro_merged.shape[1]))]].shape)
metro_merged.loc[metro_merged['Cluster Labels'] == 0, metro_merged.columns[[0] + list(range(4, metro_merged.shape[1]))]]

(108, 11)


,Name,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Preobrazhenskaya Ploshchad,Sandwich Place,Bakery,Cafeteria,Caucasian Restaurant,Breakfast Spot,Sushi Restaurant,Italian Restaurant,Pizza Place,Eastern European Restaurant,Theme Restaurant
4,Krasnoselskaya,Cafeteria,Café,Fast Food Restaurant,Restaurant,Caucasian Restaurant,Snack Place,Fried Chicken Joint,Burger Joint,Vietnamese Restaurant,Bistro
6,Krasnye Vorota,Café,Restaurant,Caucasian Restaurant,Italian Restaurant,Bakery,Cafeteria,Middle Eastern Restaurant,Deli / Bodega,Eastern European Restaurant,Irish Pub
7,Chistyye Prudy,Café,Caucasian Restaurant,Bakery,Italian Restaurant,Restaurant,Gastropub,Irish Pub,Burger Joint,Pizza Place,Modern European Restaurant
8,Lubyanka,Caucasian Restaurant,Café,Italian Restaurant,Burger Joint,Irish Pub,Modern European Restaurant,Gastropub,Eastern European Restaurant,Restaurant,Russian Restaurant
9,Okhotny Ryad,Restaurant,Caucasian Restaurant,Café,Italian Restaurant,Russian Restaurant,Seafood Restaurant,Eastern European Restaurant,Burger Joint,Asian Restaurant,Bakery
10,Biblioteka Imeni Lenina,Café,Italian Restaurant,Bakery,Restaurant,Caucasian Restaurant,Buffet,Russian Restaurant,Irish Pub,Seafood Restaurant,Pizza Place
11,Kropotkinskaya,Bakery,Café,Caucasian Restaurant,Restaurant,French Restaurant,Italian Restaurant,Steakhouse,Seafood Restaurant,Russian Restaurant,Irish Pub
12,Park Kultury,Bakery,Café,Caucasian Restaurant,Restaurant,Italian Restaurant,French Restaurant,Hot Dog Joint,Middle Eastern Restaurant,Pizza Place,Greek Restaurant
13,Frunzenskaya,Café,Bakery,Italian Restaurant,Restaurant,Pizza Place,Fast Food Restaurant,Caucasian Restaurant,Sandwich Place,Seafood Restaurant,Sushi Restaurant


Second cluster (purple) mainly consists of different fast food venues and located most far from the city center.

In [222]:
print(metro_merged.loc[metro_merged['Cluster Labels'] == 1, metro_merged.columns[[0] + list(range(4, metro_merged.shape[1]))]].shape)
metro_merged.loc[metro_merged['Cluster Labels'] == 1, metro_merged.columns[[0] + list(range(4, metro_merged.shape[1]))]]

(79, 11)


,Name,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Bulvar Rokossovskogo,Sushi Restaurant,Café,Vietnamese Restaurant,Asian Restaurant,Mexican Restaurant,Sandwich Place,Fried Chicken Joint,Fast Food Restaurant,Tex-Mex Restaurant,Food Court
18,Yugo-Zapadnaya,Fast Food Restaurant,Cafeteria,Middle Eastern Restaurant,Café,Pizza Place,Sandwich Place,Bakery,Italian Restaurant,Burger Joint,Buffet
21,Salaryevo,Fast Food Restaurant,Vietnamese Restaurant,Bistro,Restaurant,Seafood Restaurant,Eastern European Restaurant,Burger Joint,Buffet,Gastropub,Fried Chicken Joint
22,Khovrino,Restaurant,Sushi Restaurant,BBQ Joint,Shawarma Place,Deli / Bodega,Dim Sum Restaurant,Diner,Doner Restaurant,Donut Shop,Dumpling Restaurant
39,Kashirskaya,Fast Food Restaurant,Café,Italian Restaurant,Donut Shop,Restaurant,Korean Restaurant,Burger Joint,Sushi Restaurant,Blini House,Russian Restaurant
40,Kantemirovskaya,Restaurant,Pizza Place,Snack Place,Dumpling Restaurant,Bakery,Sushi Restaurant,Fried Chicken Joint,Food Truck,Fast Food Restaurant,Café
43,Domodedovskaya,Fast Food Restaurant,Café,Sushi Restaurant,Restaurant,Italian Restaurant,Blini House,Dumpling Restaurant,Middle Eastern Restaurant,Pizza Place,Japanese Restaurant
44,Krasnogvardeyskaya,Fast Food Restaurant,Asian Restaurant,Sushi Restaurant,Restaurant,Café,Bakery,Blini House,Food Court,Eastern European Restaurant,Snack Place
45,Alma-Atinskaya,Pizza Place,Fast Food Restaurant,Italian Restaurant,Sushi Restaurant,Asian Restaurant,Food Truck,Café,Middle Eastern Restaurant,Restaurant,Bakery
46,Pyatnitskoye Shosse,Pizza Place,Bakery,Sushi Restaurant,Café,Japanese Restaurant,Fast Food Restaurant,Middle Eastern Restaurant,Eastern European Restaurant,Shawarma Place,Restaurant


Third cluster (green) occupies a middle position between other clusters both  geographically and by it's content.

In [223]:
print(metro_merged.loc[metro_merged['Cluster Labels'] == 2, metro_merged.columns[[0] + list(range(4, metro_merged.shape[1]))]].shape)
metro_merged.loc[metro_merged['Cluster Labels'] == 2, metro_merged.columns[[0] + list(range(4, metro_merged.shape[1]))]]

(74, 11)


,Name,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Cherkizovskaya,Pizza Place,Fast Food Restaurant,Café,Cafeteria,Restaurant,Dumpling Restaurant,Bakery,Middle Eastern Restaurant,Burger Joint,Diner
3,Sokolniki,Café,Restaurant,Fast Food Restaurant,Eastern European Restaurant,BBQ Joint,Pizza Place,Cafeteria,Noodle House,Indian Restaurant,Russian Restaurant
5,Komsomolskaya,Café,Cafeteria,Restaurant,Bakery,Fast Food Restaurant,Middle Eastern Restaurant,Eastern European Restaurant,Italian Restaurant,Sushi Restaurant,Pizza Place
20,Rumyantsevo,Café,Snack Place,Fast Food Restaurant,Asian Restaurant,Restaurant,Italian Restaurant,Cafeteria,Blini House,Burger Joint,Chinese Restaurant
23,Belomorskaya,Pizza Place,Café,Cafeteria,Sushi Restaurant,Middle Eastern Restaurant,Caucasian Restaurant,Shawarma Place,Snack Place,Fast Food Restaurant,Vietnamese Restaurant
24,Rechnoy Vokzal,Café,Pizza Place,Sushi Restaurant,Fast Food Restaurant,Shawarma Place,Cafeteria,American Restaurant,Blini House,Middle Eastern Restaurant,Steakhouse
25,Vodny Stadion,Café,Fast Food Restaurant,Pizza Place,Shawarma Place,Middle Eastern Restaurant,Cafeteria,Sushi Restaurant,Vietnamese Restaurant,Italian Restaurant,Bakery
27,Sokol,Café,Bakery,Pizza Place,Restaurant,Sandwich Place,Shawarma Place,Fast Food Restaurant,Sushi Restaurant,Breakfast Spot,Chinese Restaurant
38,Kolomenskaya,Café,Pizza Place,Sushi Restaurant,Fast Food Restaurant,Caucasian Restaurant,Eastern European Restaurant,Donut Shop,Restaurant,Vietnamese Restaurant,Gastropub
41,Tsaritsyno,Café,Pizza Place,BBQ Joint,Shawarma Place,Fast Food Restaurant,Restaurant,Middle Eastern Restaurant,Cafeteria,Hot Dog Joint,Italian Restaurant


## Discussion.

Our analysis shows that amount of fast food restaurants changes from center of Moscow to it's outskirts. City center is crowded with regular restaurants and cafeterias and suburbs mostly filled with fast food venues. I can suggest that it happens because of high real estate prices close to the center, so low profit venues can not afford high rent. To check this hypothesis we should continue analysis, but unfortunately for today we don't have sorces of reliable data.

## Conclusion.

This reserch does not provide enaugh of valuable information for stakeholders to make a desision of opening fast food venue in Moscow, however, we made some interesting observation about relations of food venues structure and it's geographical position. Also we came up with a hypothesis that should be verifyed in future researches.